In [ ]:
import pandas as pd
import glob
import os
from collections import defaultdict
import numpy as np
import locationtagger
from pathlib import Path

USERS_DATA = r'D:/coding/classes/csci461/Rhino-Sentiment-Analysis-Project/Rhino_Users'
PROCESSED_DATA_FOLDER = r"D:/coding/classes/csci461/Rhino-Sentiment-Analysis-Project/processed_users/"

In [ ]:
#https://www.geeksforgeeks.org/extracting-locations-from-text-using-python/
#funciton to tag country


def add_tag(row):
    place = (locationtagger.find_locations(text = row.location))
    return place

def tag_country(row):
    #place = (locationtagger.find_locations(text = location))
    #print(f"{i} {place.countries} {place.regions} {place.cities} {place.country_regions}")
    place = row['place']
    if len(place.countries) != 0:
        return (place.countries[0])
    elif len(place.country_regions) != 0:
        return (list(place.country_regions.keys())[0])
    elif len(place.country_cities) != 0:
        return (list(place.country_cities.keys())[0])
    else:
        return (None)

def tag_region(row):
    place = row['place']
    if len(place.regions) != 0:
        return place.regions[0]
    elif len(place.region_cities) != 0:
        return list(place.region_cities.keys())[0]
    else:
        return None
def tag_city(row):
    place = row['place']
    if len(place.cities) != 0:
        return place.cities[0]
    else:
        return 0

   

In [ ]:
def process_df(users, filename):
    outfile_path = PROCESSED_DATA_FOLDER + filename
    users = users.dropna(axis=0, how="any")
    users["place"] = users.apply(add_tag, axis=1)
    users["country"] = users.apply(tag_country, axis=1)
    users["region"] = users.apply(tag_region, axis=1)
    users["city"] = users.apply(tag_city,axis=1)
    users.drop("place", inplace = True, axis=1)
    users.to_csv(Path(outfile_path), index=False)


In [ ]:
#take in user data
path = USERS_DATA # use your path
print(path)
all_files = glob.glob(os.path.join(path , "*.csv"))
print(os.path.join(path , "*.csv"))
li = []

for file_path in all_files:
    print(file_path)
    df = pd.read_csv(file_path, index_col=None, header=0, lineterminator='\n', converters = {'id': str})
    filename = file_path.split("\\")[-1]
    split_df =  np.array_split(df, 20)
    for i, temp in enumerate(split_df):
        temp_filename =  str(i) + filename
        print(temp_filename)
        if(os.path.isfile(PROCESSED_DATA_FOLDER + temp_filename)):
            continue
        process_df(temp, temp_filename)

